In [58]:
import json

#import distance data
with open(r'json/distance.json') as f:
    city_distances = json.load(f)

#import input data
with open(r'json/input.json') as f:
    city_data = json.load(f)


In [59]:
#function declarations 

#get distance function
def get_distance(start:str, end:str, data):
    dis = data[start][end] 
    if (type(data[start][end]) == str):
        return 0
    else:
        return dis
    

#get closest depot
def get_depot(city:str, data:dict):
    depots = ['Goderich', 'Toronto', 'Picton']
    lowest_dist = 1E9
    closest_depot = ""

    for depot in depots:
        dist = get_distance(city, depot, data)
        
        if (dist != 0 and dist < lowest_dist):
            lowest_dist = dist
            closest_depot = depot
    return closest_depot


In [60]:
class Truck:
    def __init__(self, starting_depot) -> None:
        
        # Truck constants
        self.new_truck_price = 300
        self.hourly_wage = 20
        self.overtime_wage = 30
        self.speed = 100
        self.cost_per_km = 0.68 + self.hourly_wage / self.speed
        self.cost_per_km_overtime = 0.68 + self.overtime_wage / self.speed
        
        # Truck variables
        self.home_depot = starting_depot
        self.current_city = self.home_depot
        self.rest = False
        self.distance_driven_week = 0
        self.day_cost = 0
        self.total_cost = 0
        self.packages = 5

        # Time variables
        self.current_day = "Monday"

        # Logging
        self.distance_driven_days = {
            "Monday": 0,
            "Tuesday": 0,
            "Wednesday": 0,
            "Thursday": 0,
            "Friday": 0,
            "Saturday": 0,
            "Sunday": 0,
        }
        self.costs_days = {
            "Monday": 0,
            "Tuesday": 0,
            "Wednesday": 0,
            "Thursday": 0,
            "Friday": 0,
            "Saturday": 0,
            "Sunday": 0
        }
        self.deliveries_info = {
            "Monday": [],
            "Tuesday": [],
            "Wednesday": [],
            "Thursday": [],
            "Friday": [],
            "Saturday": [],
            "Sunday": []
        }

    def generate_day_info(self, weekday) -> None:
        day_info = {
                "Total_distance": self.distance_driven_days[weekday],
                "Total_cost": self.costs_days[weekday],
        }
        i = 1
        for delivery in self.deliveries_info[weekday]:
            day_info[f'Delivery_{i}'] = delivery
            i += 1
        
        return day_info

    def generate_delivery_info(self, start_t, end_t, start_city, end_city, delivery_type, start_quant, end_quant, city_distances) -> dict:
        delivery_info = {
            "Start_time": start_t,
            "End_time": end_t,
            "Start_destination":start_city,
            "End_destination":end_city,
            "Distance": get_distance(start_city, end_city, city_distances),
            "Delivery_type":delivery_type,
            "Start_quantity":start_quant,
            "End_quantity": end_quant,
        }
        return delivery_info

    def generate_truck_output(self) -> dict:
        output_dict = {
            "Total_cost": self.total_cost
        }
        weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        for weekday in weekdays:
            output_dict[weekday] = self.generate_day_info(weekday)

        return output_dict

    def calculate_cost_from_km(self, distance, overtime:bool):
        rate = self.cost_per_km 
        if overtime:
            rate = self.cost_per_km_overtime
        return distance * self.cost_per_km
    
    def update_total_cost(self):
        self.total_cost = sum(self.costs_days.values()) + self.new_truck_price
    
    def move_truck(self, new_city:str, city_distances:dict, city_data:dict, weekday:str, deliver:bool=True):
        # Calculate cost
        dist = get_distance(self.current_city, new_city, city_distances)
        trip_cost = self.calculate_cost_from_km(dist, False)
        self.costs_days[weekday] += trip_cost

        # Update truck 
        self.update_total_cost()
        old_city = self.current_city
        self.distance_driven_days[weekday] += dist
        self.current_city = new_city
        
        # Deliver packages 
        start_packages = self.packages
        if deliver:
            num_packages_req = city_data[new_city]
            if self.packages <= num_packages_req:
                city_data[new_city] -= self.packages
                self.packages = 0
            else:
                self.packages -= city_data[new_city]
                city_data[new_city] = 0
        
        # Load up on packages 
        else:
            self.packages = 5
        
        # Create delivery log
        delivery_type = "Delivery"
        if not deliver:
            delivery_type = "Re-loading"
        
        self.deliveries_info[self.current_day].append(self.generate_delivery_info(0, 0, old_city, new_city, delivery_type, start_packages, self.packages, city_distances))

        #return (self.generate_delivery_info(0, 0, old_city, new_city, delivery_type, start_packages, self.packages, city_distances))
            

    def kill_trucker(self) -> None:
        self.rest = True

In [61]:
#main algorithm
Trucks = []

for i, city in enumerate(city_data):
    closest = get_depot(city, city_distances)
    while (city_data[city] > 0):
        new_truck = Truck(closest)
        new_truck.move_truck(city, city_distances, city_data, "Monday")
        Trucks.append(new_truck)

In [62]:
#outputing to file
final_total_cost = 0
for truck in Trucks:
    final_total_cost += truck.total_cost

output_dictionary = {
    "Total Cost": final_total_cost
}
i = 0
for truck in Trucks:
    output_dictionary[f'Truck_{i}'] = truck.generate_truck_output()
    i += 1

output_dictionary

{'Total Cost': 10784.320000000002,
 'Truck_0': {'Total_cost': 483.04,
  'Monday': {'Total_distance': 208,
   'Total_cost': 183.04000000000002,
   'Delivery_1': {'Start_time': 0,
    'End_time': 0,
    'Start_destination': 'Goderich',
    'End_destination': 'Tilbury',
    'Distance': 208,
    'Delivery_type': 'Delivery',
    'Start_quantity': 5,
    'End_quantity': 0}},
  'Tuesday': {'Total_distance': 0, 'Total_cost': 0},
  'Wednesday': {'Total_distance': 0, 'Total_cost': 0},
  'Thursday': {'Total_distance': 0, 'Total_cost': 0},
  'Friday': {'Total_distance': 0, 'Total_cost': 0},
  'Saturday': {'Total_distance': 0, 'Total_cost': 0},
  'Sunday': {'Total_distance': 0, 'Total_cost': 0}},
 'Truck_1': {'Total_cost': 483.04,
  'Monday': {'Total_distance': 208,
   'Total_cost': 183.04000000000002,
   'Delivery_1': {'Start_time': 0,
    'End_time': 0,
    'Start_destination': 'Goderich',
    'End_destination': 'Tilbury',
    'Distance': 208,
    'Delivery_type': 'Delivery',
    'Start_quantity'

In [63]:
output_dictionary = json.dumps(output_dictionary, indent=4)

with open(r"json/output.json", "w") as outfile:
    outfile.write(output_dictionary)